<a href="https://colab.research.google.com/github/GauravSahani1417/Covid-19-News-Classifier-and-Symtom-Predictor/blob/master/Covid_19_Analysis_%26_FbProphet_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
Covid_Summary = pd.read_csv('/content/drive/MyDrive/Covid-19 Analysis/covid_19_india.csv')
Covid_Summary.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed
0,1,2020-01-30,6:00 PM,Kerala,1,0,0,0,1
1,2,2020-01-31,6:00 PM,Kerala,1,0,0,0,1
2,3,2020-02-01,6:00 PM,Kerala,2,0,0,0,2
3,4,2020-02-02,6:00 PM,Kerala,3,0,0,0,3
4,5,2020-02-03,6:00 PM,Kerala,3,0,0,0,3


In [ ]:
Covid_Summary['Date'].max()

'2021-05-08'

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
Covid_Summary.isnull().sum()

Sno                         0
Date                        0
Time                        0
State/UnionTerritory        0
ConfirmedIndianNational     0
ConfirmedForeignNational    0
Cured                       0
Deaths                      0
Confirmed                   0
dtype: int64

In [ ]:
Covid_Summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14690 entries, 0 to 14689
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Sno                       14690 non-null  int64 
 1   Date                      14690 non-null  object
 2   Time                      14690 non-null  object
 3   State/UnionTerritory      14690 non-null  object
 4   ConfirmedIndianNational   14690 non-null  object
 5   ConfirmedForeignNational  14690 non-null  object
 6   Cured                     14690 non-null  int64 
 7   Deaths                    14690 non-null  int64 
 8   Confirmed                 14690 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 1.0+ MB


In [ ]:
Covid_Summary = Covid_Summary.drop(['Sno','ConfirmedIndianNational', 'ConfirmedForeignNational'], axis=1)
Covid_Summary.head()

,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed
0,2020-01-30,6:00 PM,Kerala,0,0,1
1,2020-01-31,6:00 PM,Kerala,0,0,1
2,2020-02-01,6:00 PM,Kerala,0,0,2
3,2020-02-02,6:00 PM,Kerala,0,0,3
4,2020-02-03,6:00 PM,Kerala,0,0,3


In [ ]:
Covid_Summary['Active'] = Covid_Summary['Confirmed'] - Covid_Summary['Cured'] - Covid_Summary['Deaths']
Covid_Summary.tail()

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


,Date,Time,State/UnionTerritory,Cured,Deaths,Confirmed,Active
14685,2021-05-08,8:00 AM,Telangana,413225,2666,487199,71308
14686,2021-05-08,8:00 AM,Tripura,34270,405,37208,2533
14687,2021-05-08,8:00 AM,Uttarakhand,158872,3430,229993,67691
14688,2021-05-08,8:00 AM,Uttar Pradesh,1184688,14873,1453679,254118
14689,2021-05-08,8:00 AM,West Bengal,818108,12076,954282,124098


In [ ]:
Covid_Summary['Date'] = pd.to_datetime(Covid_Summary['Date'])

In [ ]:
Covid_Summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14690 entries, 0 to 14689
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  14690 non-null  datetime64[ns]
 1   Time                  14690 non-null  object        
 2   State/UnionTerritory  14690 non-null  object        
 3   Cured                 14690 non-null  int64         
 4   Deaths                14690 non-null  int64         
 5   Confirmed             14690 non-null  int64         
 6   Active                14690 non-null  int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 803.5+ KB


In [ ]:
india_cases = Covid_Summary[Covid_Summary['Date'] == Covid_Summary['Date'].max()].copy().fillna(0)
india_cases.index = india_cases["State/UnionTerritory"]
india_cases = india_cases.drop(['State/UnionTerritory', 'Time','Date'], axis=1)

In [ ]:
india_cases.head()

,Cured,Deaths,Confirmed,Active
State/UnionTerritory,,,,
Andaman and Nicobar Islands,6030,72,6311,209
Andhra Pradesh,1050160,8519,1245374,186695
Arunachal Pradesh,17849,60,19802,1893
Assam,246242,1578,283313,35493
Bihar,449063,3139,567269,115067


In [ ]:
df = pd.DataFrame(pd.to_numeric(india_cases.sum())).transpose()
df.style.background_gradient(cmap='BuGn',axis=1)

,Cured,Deaths,Confirmed,Active
0,17930960,238270,21892676,3723446


In [ ]:
Trend = Covid_Summary.groupby(['Date'])['Confirmed', 'Deaths','Cured',].sum().reset_index()

In [ ]:
Trend.head()

,Date,Confirmed,Deaths,Cured
0,2020-01-30,1,0,0
1,2020-01-31,1,0,0
2,2020-02-01,2,0,0
3,2020-02-02,3,0,0
4,2020-02-03,3,0,0


In [ ]:
fig = go.Figure(go.Bar(x= Trend.Date, y= Trend.Cured, name='Recovered'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Deaths, name='Deaths'))
fig.add_trace(go.Bar(x=Trend.Date, y= Trend.Confirmed, name='Confirmed'))

fig.update_layout(barmode='stack',legend_orientation="h",legend=dict(x= 0.3, y=1.1),
                 paper_bgcolor='white',
                 plot_bgcolor = "white",)
fig.show()

In [ ]:
import plotly.express as px

In [ ]:
def horizontal_bar_chart(df, x, y, title, x_label, y_label, color):
    fig = px.bar(df, x=x, y=y, orientation='h', title=title, 
                 labels={x.name: x_label,
                         y.name: y_label}, color_discrete_sequence=[color])
    fig.update_layout(yaxis={'categoryorder': 'total ascending'})
    fig.show()

In [ ]:
top_10_death_states = india_cases.sort_values('Deaths',ascending = False)[:10]

horizontal_bar_chart(top_10_death_states,top_10_death_states.Deaths, top_10_death_states.index,
                     'Top 10 States with most deaths','Number of deaths(In Thousands)','State Name','Orange')

In [ ]:
top_10_confirmed_states = india_cases.sort_values('Confirmed', ascending=False)[:10]

horizontal_bar_chart(top_10_confirmed_states,top_10_confirmed_states.Confirmed, top_10_confirmed_states.index,
            'Top 10 Indian States (Confirmed Cases)', 'Number of Confirmed cases (in Thousands)','States Name','blue')

In [ ]:
top_10_recovered_states = india_cases.sort_values('Cured', ascending=False)[:10]

horizontal_bar_chart(top_10_recovered_states, top_10_recovered_states.Cured, top_10_recovered_states.index,
                    'Top 10 States (Cured Cases)', 'Number of Cured cases (in Thousands)', 'States Name', 'Purple')

In [ ]:
vaccination = pd.read_csv('/content/drive/MyDrive/Covid-19 Analysis/covid_vaccine_statewise.csv')

In [ ]:
vaccination.tail()

,Updated On,State,Total Individuals Vaccinated,Total Sessions Conducted,Total Sites,First Dose Administered,Second Dose Administered,Male(Individuals Vaccinated),Female(Individuals Vaccinated),Transgender(Individuals Vaccinated),Total Covaxin Administered,Total CoviShield Administered,AEFI,18-30 years (Age),30-45 years (Age),45-60 years (Age),60+ years (Age),Total Doses Administered
4250,06/05/2021,West Bengal,8777268.0,301115.0,1674.0,8777268.0,2817455.0,4715269.0,4060997.0,1002.0,1226791.0,10367932.0,1028.0,275491.0,800454.0,3957788.0,3742778.0,11594723.0
4251,07/05/2021,West Bengal,8808010.0,512145.0,2940.0,8808010.0,3031670.0,4732280.0,4074724.0,1006.0,1246103.0,10593577.0,1030.0,277583.0,804206.0,3969145.0,3756318.0,11839680.0
4252,08/05/2021,West Bengal,8823763.0,341101.0,2027.0,8823763.0,3128641.0,4741259.0,4081493.0,1011.0,1260663.0,10691741.0,1031.0,279199.0,807324.0,3974768.0,3761713.0,11952404.0
4253,09/05/2021,West Bengal,8825922.0,100240.0,557.0,8825922.0,3137939.0,4742419.0,4082492.0,1011.0,1262952.0,10700909.0,1031.0,279322.0,807619.0,3975640.0,3762582.0,11963861.0
4254,10/05/2021,West Bengal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
vaccination['Total Vaccinatons'] = vaccination['First Dose Administered']+vaccination['Second Dose Administered']

#Renaming columns
vaccination.rename(columns = {'Updated On':'Date'}, inplace = True)

In [ ]:
Maharashtra = vaccination[vaccination["State"]=="Maharashtra"]

fig = px.line(Maharashtra,x="Date",y="Total Vaccinatons",title="Vaccination till date in Maharashtra")  
fig.update_xaxes(rangeslider_visible=True) 

In [ ]:
fig = px.line(Maharashtra,x="Date",y="Total CoviShield Administered",title="CoviShield Administered in Mahrashtra")  

fig.update_xaxes(rangeslider_visible=True) 

In [ ]:
fig = px.line(Maharashtra,x="Date",y="Total Covaxin Administered",title="Covaxin Administered in Maharashtra") 
 
fig.update_xaxes(rangeslider_visible=True) 

In [ ]:
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
from plotly.offline import iplot, init_notebook_mode

In [ ]:
model = Prophet()

In [ ]:
Confirmed = Trend.loc[:, ['Date', 'Confirmed']] 
Confirmed.tail()

,Date,Confirmed
460,2021-05-04,20282833
461,2021-05-05,20665148
462,2021-05-06,21077410
463,2021-05-07,21491598
464,2021-05-08,21892676


In [ ]:
Confirmed.columns = ['ds', 'y']

model.fit(Confirmed)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [ ]:
future = model.make_future_dataframe(periods=60) # helper function to extend the dataframe for specified days

future.tail()

,ds
520,2021-07-03
521,2021-07-04
522,2021-07-05
523,2021-07-06
524,2021-07-07


In [ ]:
forecast_india_conf = model.predict(future)
forecast_india_conf

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-30,-1.202109e+05,-1.069741e+06,9.266173e+05,-1.202109e+05,-1.202109e+05,9692.010122,9692.010122,9692.010122,9692.010122,9692.010122,9692.010122,0.0,0.0,0.0,-1.105189e+05
1,2020-01-31,-1.178167e+05,-1.139628e+06,8.549295e+05,-1.178167e+05,-1.178167e+05,23105.062922,23105.062922,23105.062922,23105.062922,23105.062922,23105.062922,0.0,0.0,0.0,-9.471168e+04
2,2020-02-01,-1.154226e+05,-1.077179e+06,1.001325e+06,-1.154226e+05,-1.154226e+05,37679.213841,37679.213841,37679.213841,37679.213841,37679.213841,37679.213841,0.0,0.0,0.0,-7.774334e+04
3,2020-02-02,-1.130284e+05,-1.084944e+06,8.974227e+05,-1.130284e+05,-1.130284e+05,-29289.227574,-29289.227574,-29289.227574,-29289.227574,-29289.227574,-29289.227574,0.0,0.0,0.0,-1.423176e+05
4,2020-02-03,-1.106342e+05,-1.134745e+06,7.765856e+05,-1.106342e+05,-1.106342e+05,-20258.642988,-20258.642988,-20258.642988,-20258.642988,-20258.642988,-20258.642988,0.0,0.0,0.0,-1.308928e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
520,2021-07-03,2.082336e+07,1.969812e+07,2.208970e+07,2.009171e+07,2.146029e+07,37679.213841,37679.213841,37679.213841,37679.213841,37679.213841,37679.213841,0.0,0.0,0.0,2.086104e+07
521,2021-07-04,2.089561e+07,1.957927e+07,2.214577e+07,2.014677e+07,2.155371e+07,-29289.227574,-29289.227574,-29289.227574,-29289.227574,-29289.227574,-29289.227574,0.0,0.0,0.0,2.086632e+07
522,2021-07-05,2.096787e+07,1.959213e+07,2.225627e+07,2.020874e+07,2.165278e+07,-20258.642988,-20258.642988,-20258.642988,-20258.642988,-20258.642988,-20258.642988,0.0,0.0,0.0,2.094761e+07
523,2021-07-06,2.104012e+07,1.971631e+07,2.228217e+07,2.025586e+07,2.173719e+07,-15635.830830,-15635.830830,-15635.830830,-15635.830830,-15635.830830,-15635.830830,0.0,0.0,0.0,2.102448e+07


In [ ]:
fig = plot_plotly(model, forecast_india_conf) 

fig.update_layout(template='plotly_white')

iplot(fig) 